In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import talib as ta

In [2]:
sp500_df = pd.read_csv('./data/Stocks in the SP 500 Index.csv')
sp500_df.head(3)

,Symbol,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Price to TTM earnings,Price to TTM sales,Price to book value,Action
0,AAPL,Apple Inc,Common stocks,Large cap,Information Technology,"$2,815,919,544,550",0.51%,27.41,7.44,38.31,Analyze
1,MSFT,Microsoft Corp,Common stocks,Large cap,Information Technology,"$2,245,311,495,186",0.79%,30.62,12.14,13.62,Analyze
2,GOOG,Alphabet Inc Class C,Common stocks,Large cap,Communication Services,"$1,856,551,479,248",0.00%,10.99,7.21,3.32,Analyze


In [3]:
def add_ratio1 (df):
    df['Ratio1']=((df['Close']+df['Open'])/2-df['Low'])/(df['High']-df['Low'])

def add_ratio2 (df):
    df['Ratio2']=(df['Close']-df['Open'])/(df['High']-df['Low'])

def add_avg (df):
    df['avg']=(df['Open']+df['Close']+df['High']+df['Low'])/4

def add_trend_tmr_up (df):
    df['avg_tmr']=np.concatenate((df['avg'].to_numpy()[1:],[None]))
    df['trend_tmr_up'] = np.where(df['avg_tmr']-df['avg']>0, True, False)
    df.dropna(inplace=True)
    del df['avg_tmr']

In [15]:
def sp_500_stocks (timeframe):
    df_all=pd.DataFrame()
    #for i in sp500_df.Symbol:
    for i in sp500_df.Symbol[0:1]:
        stk = yf.Ticker(i)
        df = stk.history(period=timeframe)
        add_ratio1(df)
        add_ratio2(df)
        add_avg (df)
        add_trend_tmr_up (df)
        df['Name']=i
        df_all=pd.concat([df_all, df])
    return df_all

In [16]:
x=sp_500_stocks('2y')

In [17]:
x

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ratio1,Ratio2,avg,trend_tmr_up,Name
Date,,,,,,,,,,,,
2020-02-24,73.307668,75.014214,71.327380,73.534546,222195200,0.0,0.0,0.567893,0.061537,73.295952,False,AAPL
2020-02-25,74.217663,74.607306,70.562882,71.043770,230673600,0.0,0.0,0.511280,-0.784758,72.607905,False,AAPL
2020-02-26,70.661532,73.460571,70.654134,72.170792,198054800,0.0,0.0,0.271529,0.537785,71.736757,False,AAPL
2020-02-27,69.322419,70.530813,67.315000,67.453102,320605600,0.0,0.0,0.333589,-0.581289,68.655334,False,AAPL
2020-02-28,63.443215,68.659042,63.223728,67.413651,426510000,0.0,0.0,0.405626,0.730489,65.684909,True,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-14,167.369995,169.580002,166.559998,168.880005,86185500,0.0,0.0,0.518212,0.500003,168.097500,True,AAPL
2022-02-15,170.970001,172.949997,170.250000,172.789993,64286300,0.0,0.0,0.603703,0.674072,171.739998,True,AAPL
2022-02-16,171.850006,173.339996,170.050003,172.550003,61177400,0.0,0.0,0.653497,0.212765,171.947502,False,AAPL


In [18]:
x['MA']=ta.MA(x.Close)
x

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ratio1,Ratio2,avg,trend_tmr_up,Name,MA
Date,,,,,,,,,,,,,
2020-02-24,73.307668,75.014214,71.327380,73.534546,222195200,0.0,0.0,0.567893,0.061537,73.295952,False,AAPL,NaN
2020-02-25,74.217663,74.607306,70.562882,71.043770,230673600,0.0,0.0,0.511280,-0.784758,72.607905,False,AAPL,NaN
2020-02-26,70.661532,73.460571,70.654134,72.170792,198054800,0.0,0.0,0.271529,0.537785,71.736757,False,AAPL,NaN
2020-02-27,69.322419,70.530813,67.315000,67.453102,320605600,0.0,0.0,0.333589,-0.581289,68.655334,False,AAPL,NaN
2020-02-28,63.443215,68.659042,63.223728,67.413651,426510000,0.0,0.0,0.405626,0.730489,65.684909,True,AAPL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-14,167.369995,169.580002,166.559998,168.880005,86185500,0.0,0.0,0.518212,0.500003,168.097500,True,AAPL,170.679710
2022-02-15,170.970001,172.949997,170.250000,172.789993,64286300,0.0,0.0,0.603703,0.674072,171.739998,True,AAPL,170.380096
2022-02-16,171.850006,173.339996,170.050003,172.550003,61177400,0.0,0.0,0.653497,0.212765,171.947502,False,AAPL,170.149385


In [6]:
print(x.shape)
x.head(5)

(253670, 13)


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ratio1,Ratio2,avg,trend_tmr_up,Name,Adj Close
Date,,,,,,,,,,,,,
2020-02-19,78.915603,80.042618,78.915603,79.808334,93984000.0,0.0,0.0,0.396060,0.792120,79.420540,False,AAPL,NaN
2020-02-20,79.564202,80.062354,78.474177,78.989594,100566000.0,0.0,0.0,0.505435,-0.361804,79.272582,False,AAPL,NaN
2020-02-21,78.575295,79.026598,76.572812,77.201668,129554000.0,0.0,0.0,0.536179,-0.559799,77.844093,False,AAPL,NaN
2020-02-24,73.307668,75.014214,71.327380,73.534546,222195200.0,0.0,0.0,0.567893,0.061537,73.295952,False,AAPL,NaN
2020-02-25,74.217671,74.607314,70.562890,71.043777,230673600.0,0.0,0.0,0.511280,-0.784758,72.607913,False,AAPL,NaN


In [ ]:
x.to_csv('./data/sp500_2y.csv')